In [1]:
import numpy as np
import cv2

In [2]:
# load serialized model from disk
net = cv2.dnn.readNetFromCaffe('source/deploy.prototxt.txt', 
                               'source/res10_300x300_ssd_iter_140000.caffemodel')

# load the input image & copy it
image = cv2.imread('me-anas.jpg')
imgCopy = image.copy()
h, w = image.shape[:2]
# construct an input blob for the image, resize, and normalise
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, 
                             (300, 300), (104.0, 177.0, 123.0))

In [3]:
# pass the blob through the model to get detections & predictions
net.setInput(blob)
detections = net.forward()

In [ ]:
# define the minimum confidence
min_confidence = 0.7

# loop over the detections
for i in range(detections.shape[2]):
    # extract the confidence (i.e., probability) associated with the prediction
    confidence = detections[0, 0, i, 2]
    
    # filter out weak detections by ensuring the `confidence` is > the minimum confidence
    if confidence > min_confidence:
        # compute the (x, y)-coordinates of the bounding box for the object
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype('int')
        
        crop_img = image[startY:endY, startX: endX]
        cv2.imwrite('croped'+ str(i)+'.jpg', crop_img)
        # draw the bounding box of the face along with the associated probability
        text = "{:.2f}%".format(confidence * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(imgCopy, (startX, startY), (endX, endY), (0, 0, 255), 2)
        cv2.putText(imgCopy, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        
# show the output image
cv2.imshow("Output", imgCopy)
cv2.imshow("cropped", crop_img)
cv2.waitKey(0)
cv2.destroyAllWindows()